In [ ]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

def get_tweet_data(card):
    username = card.find_element(By.XPATH, './/div[@dir="ltr"]').text
    try:
        handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    try:
        post_date = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment1 = card.find_element(By.XPATH, './/div[2]/div[2]/div[1]').text
    comment2 = card.find_element(By.XPATH, './/div[2]/div[2]/div[2]').text
    tweet_text = comment1 + comment2
    comment_count = card.find_element(By.XPATH, './/div[@data-testid="reply"]//span').text
    retweet_count = card.find_element(By.XPATH, './/div[@data-testid="retweet"]//span').text
    like_count = card.find_element(By.XPATH, './/div[@data-testid="like"]//span').text

    tweet = (username, handle, post_date, tweet_text, comment_count, retweet_count, like_count)
    return tweet
    
    
    
driver_path = 'C:\\Users\\chide\\Desktop\\AI sentiment analysis project\\edgedriver_win64\\msedgedriver.exe'

options = EdgeOptions()
options.use_chromium = True
driver = Edge(executable_path=driver_path, options=options)



driver.get('https://twitter.com/login')
driver.maximize_window()
sleep(5)

uname = input('Input Username: ')
username = driver.find_element(By.XPATH,'//input[@autocomplete="username"]')
username.send_keys(uname)
username.send_keys(Keys.RETURN)
sleep(5)

pword = input('Input Password: ')
password = driver.find_element(By.NAME, 'password')
password.send_keys(pword)
password.send_keys(Keys.RETURN)
sleep(10)


search_term = input('Input Search term: ')
search_input = driver.find_element(By.XPATH, '//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(5)

driver.find_element(By.LINK_TEXT, 'Latest').click()

data = []

tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(3)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(20) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

